In [10]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
base_dir = 'airportsecurity/datasets'
train_dir = 'D:/GitHub/airportsecurity/datasets/train'
valid_dir = 'D:/GitHub/airportsecurity/datasets/valid'
test_dir = 'D:/GitHub/airportsecurity/datasets/test'


In [12]:
# Read CSV files
train_df = pd.read_csv('D:/GitHub/airportsecurity/datasets/train/_annotations_train.csv')
valid_df = pd.read_csv('D:/GitHub/airportsecurity/datasets/valid/_annotations_valid.csv')
test_df = pd.read_csv('D:/GitHub/airportsecurity/datasets/test/_annotations_test.csv')

In [13]:
# Data preprocessing
image_size = (224,224)
batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=train_dir,
    x_col='filename',
    y_col=['xmin', 'ymin', 'xmax', 'ymax', 'class'],
    target_size=image_size,
    batch_size=batch_size,
    class_mode='other')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory=valid_dir,
    x_col='filename',
    y_col=['xmin', 'ymin', 'xmax', 'ymax', 'class'],
    target_size=image_size,
    batch_size=batch_size,
    class_mode='other')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='filename',
    y_col=['xmin', 'ymin', 'xmax', 'ymax', 'class'],
    target_size=image_size,
    batch_size=batch_size,
    class_mode='other')


Found 6181 validated image filenames.
Found 1766 validated image filenames.
Found 883 validated image filenames.


In [24]:
'''from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Reshape, Activation

# Load pre-trained MobileNetV2 model (without top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom detection head
x = base_model.output
x = Conv2D(256, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(5, (1, 1), activation='sigmoid')(x)  # Output layer: 5 channels (4 for bounding box coordinates, 1 for class probability)
# Change the reshape operation to match the expected output shape during training
x = Reshape((7, 7, 5))(x)  # Reshape to (7, 7, 5) for anchor boxes (or bounding boxes) and class probability

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])'''

import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model

# Define your base model (e.g., MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define the convolutional layers for prediction
x = base_model.output
x = Conv2D(256, (3, 3), padding='same')(x)
x = Activation('relu')(x)
x = Conv2D(5, (1, 1), activation='sigmoid')(x)  # Output layer: 5 channels (4 for bounding box coordinates, 1 for class probability)

# Create the model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])



In [25]:
# Assuming you have prepared your data and defined your fully convolutional object detection model (model variable)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)


Epoch 1/10


ValueError: Dimensions must be equal, but are 5 and 7 for '{{node Equal}} = Equal[T=DT_INT64, incompatible_shape_error=true](data_1, Cast_2)' with input shapes: [?,5], [?,7,7].